# Data pre-processing for dublin-bus datasets

In [3]:
import os
import sys
import pandas as pd
import progressbar
import json

## Read data into a dataframe

In [4]:
header = ['Timestamp','Line_ID','Direction','Journey_Pattern_ID','Time_Frame','Vehicle_Journey_ID', \
         'Operator','Congestion','Lon','Lat','Delay','Block_ID','Vehicle_ID','Stop_ID','At_Stop']
data_dir = '../dublin_data/'
data_file = data_dir + 'siri.20130101.csv'
df = pd.read_csv(data_file, header=None, names=header)

In [5]:
items = os.listdir(data_dir)
items = [item for item in items if (item[:4]=='siri')]
# print (items)

In [6]:
# print df (first 10 rows)
df[:10]

,Timestamp,Line_ID,Direction,Journey_Pattern_ID,Time_Frame,Vehicle_Journey_ID,Operator,Congestion,Lon,Lat,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop
0,1356998403000000,747.0,0,07470001,2012-12-31,3493,SL,0,-6.236852,53.425327,-709,747006,40040,7411.0,0
1,1356998405000000,27.0,0,NaN,2012-12-31,3883,RD,0,-6.233417,53.342232,0,27017,33521,395.0,0
2,1356998407000000,40.0,0,NaN,2012-12-31,2226,HN,0,-6.278250,53.416683,0,40206,33142,6071.0,0
3,1356998407000000,7.0,0,00071003,2012-12-31,6106,D1,0,-6.231633,53.317768,0,7019,43004,3222.0,1
4,1356998411000000,747.0,0,07471001,2012-12-31,3531,SL,0,-6.254617,53.355484,-454,747007,40039,1445.0,0
5,1356998411000000,56.0,0,056A1001,2012-12-31,1830,RD,0,-6.233183,53.342201,0,56001,33488,2379.0,0
6,1356998417000000,25.0,0,025A0001,2012-12-31,2866,CD,0,-6.296867,53.347500,0,25007,33604,4604.0,0
7,1356998423000000,747.0,0,07470001,2012-12-31,3493,SL,0,-6.238668,53.425789,-687,747006,40040,7411.0,0
8,1356998425000000,27.0,0,NaN,2012-12-31,3883,RD,0,-6.233400,53.342232,0,27017,33521,395.0,0
9,1356998427000000,4.0,0,NaN,2012-12-31,4243,HN,0,-6.279000,53.416683,0,4001,43043,7226.0,0


## Main functions

In [17]:
# output a dataframe to specific file
def output(df,outfile):
    df.to_csv(outfile, index=False)

# keep columns by column names
def keep_cols(df,cols):
    drops = header[:]
    [drops.remove(i) for i in cols]
    ndf = df.drop(drops, axis=1)
    return ndf

# keep rows by condition
def keep_rows(df,col,satisfy,threshold):
    ndf = df[satisfy(df,col,threshold)]
    return ndf

def line2json(files):
    lines = dict()
    for data_file in files:
        df = pd.read_csv(data_dir + data_file, header=None, names=header)
        ndf = df[~df.Line_ID.isnull()]
        line_ids = [str(i) for i in set(ndf['Line_ID'].values)]
        lines[data_file] = line_ids
        print (lines)
    return lines

In [8]:
# Return true if col value equal or larger then threshold
def larger(df,col,threshold=1):
    return df[col] >= threshold

## Tests

In [9]:
# test keep_column
cols = ['Timestamp','Line_ID','Lon','Lat','Vehicle_ID']
ndf = keep_cols(df,cols)
nndf = ndf[~ndf.Line_ID.isnull()]
line_ids = set(nndf['Line_ID'].values)
ldf = []
line = dict()
for line_id in line_ids:
    sndf = ndf[ndf.Line_ID == line_id]
    vehicle_ids = set(sndf['Vehicle_ID'].values)
    
    # For a specific line_id, find the vehicle_id has largest data point
    for vehicle_id in vehicle_ids:
        tsndf = sndf[sndf.Vehicle_ID == vehicle_id]
        if line_id in line.keys():
            if line[line_id].shape[0] < tsndf.shape[0]:
                line[line_id] = tsndf
        else:
            line[line_id] = tsndf
    # Make lines (from points)
    lsndf = sndf.copy()
    sndf.drop(sndf.index[-1],inplace=True)
    lsndf.drop(lsndf.index[0],inplace=True)
    sndf['toLon'] = lsndf['Lon'].values
    sndf['toLat'] = lsndf['Lat'].values

# Combine all lines to a dataframe
for line_id in line.keys():
    ldf.append(line[line_id])
odf = pd.concat(ldf)


/Users/haonanl5/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/Users/haonanl5/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/haonanl5/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-

In [18]:
def line_stop_summary(files):
    ldf = list()
    for data_file in files:
        df = pd.read_csv(data_dir + data_file, header=None, names=header)
        cols = ['Lon','Lat','At_Stop', 'Line_ID']
        ndf = keep_cols(df,cols)
        nndf = keep_rows(ndf,'At_Stop',larger,1)
        ldf.append(nndf)
    odf = pd.concat(ldf)
    lines = set(odf['Line_ID'].values)
    for line in lines:
        
    return odf

In [19]:
    files = os.listdir(data_dir)
    files = [item for item in files if (item[:4]=='siri')]
    odf = line_stop_summary(files)

In [44]:
lines = set(odf['Line_ID'].values)
line = 1.0
ndf = odf[odf['Line_ID']==line]
x = ndf.drop_duplicates()

In [61]:
ndf[ndf.duplicated(keep=False)]

,Line_ID,Lon,Lat,At_Stop
40576,1.0,-6.259525,53.348591,1
43025,1.0,-6.207619,53.325043,1
43597,1.0,-6.256140,53.345970,1
44294,1.0,-6.208836,53.328621,1
45341,1.0,-6.210115,53.331657,1
47253,1.0,-6.245327,53.343327,1
50170,1.0,-6.223919,53.340126,1
50855,1.0,-6.223919,53.340126,1
51246,1.0,-6.223919,53.340126,1
51726,1.0,-6.223919,53.340126,1


In [60]:
t.shape

(142736, 4)